In [1]:
import geopandas as gpd
from shapely.geometry import mapping
from osgeo import gdal, ogr


In [35]:
import pyproj
import pandas as pd

# 加载DEM栅格数据
dem_path = 'rain_raster.tif'
dataset = gdal.Open(dem_path)

# 获取DEM栅格的宽度和高度
cols = dataset.RasterXSize
rows = dataset.RasterYSize

# 获取DEM栅格的地理转换信息
transform = dataset.GetGeoTransform()

# 创建投影转换器
src_proj = pyproj.Proj(dataset.GetProjection())
target_proj = pyproj.Proj(init='EPSG:4326')  # WGS84坐标系
transformer = pyproj.Transformer.from_proj(src_proj, target_proj)

# 计算像元的经度和纬度
lon_values = []
lat_values = []

for i in range(cols):
    for j in range(rows):
        # 计算像元的中心坐标
        x = transform[0] + (i + 0.5) * transform[1] + (j + 0.5) * transform[2]
        y = transform[3] + (i + 0.5) * transform[4] + (j + 0.5) * transform[5]

        # 进行坐标转换
        lon, lat = transformer.transform(x, y)

        lon_values.append(lon)
        lat_values.append(lat)

# 获取DEM栅格的海拔值
band = dataset.GetRasterBand(1)
elevation_values = band.ReadAsArray().flatten()

# 创建DataFrame
df = pd.DataFrame({'Longitude': lon_values, 'Latitude': lat_values, 'rain': elevation_values})

# 关闭数据集
dataset = None
df = df[df['rain'] != 0]
# 打印DataFrame

df.to_csv("降水.csv")
df


C:\Users\lenovo\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Longitude,Latitude,rain
0,108.167227,33.742821,472.906342
1,108.167227,33.741148,484.011139
2,108.167227,33.739474,495.804657
3,108.167227,33.737801,508.204315
4,108.167227,33.736128,521.101562
...,...,...,...
30271,108.511317,33.736128,483.156860
30272,108.511317,33.734454,480.930298
30273,108.511317,33.732781,479.297668
30274,108.511317,33.731107,478.129822


In [36]:
dfm=pd.read_csv("降水.csv")

In [37]:
# 加载DEM栅格数据
dem_path = 'tem_raster.tif'
dataset = gdal.Open(dem_path)

# 获取DEM栅格的宽度和高度
cols = dataset.RasterXSize
rows = dataset.RasterYSize

# 获取DEM栅格的地理转换信息
transform = dataset.GetGeoTransform()

# 创建投影转换器
src_proj = pyproj.Proj(dataset.GetProjection())
target_proj = pyproj.Proj(init='EPSG:4326')  # WGS84坐标系
transformer = pyproj.Transformer.from_proj(src_proj, target_proj)

# 计算像元的经度和纬度
lon_values = []
lat_values = []

for i in range(cols):
    for j in range(rows):
        # 计算像元的中心坐标
        x = transform[0] + (i + 0.5) * transform[1] + (j + 0.5) * transform[2]
        y = transform[3] + (i + 0.5) * transform[4] + (j + 0.5) * transform[5]

        # 进行坐标转换
        lon, lat = transformer.transform(x, y)

        lon_values.append(lon)
        lat_values.append(lat)

# 获取DEM栅格的海拔值
band = dataset.GetRasterBand(1)
elevation_values = band.ReadAsArray().flatten()

# 创建DataFrame
df = pd.DataFrame({'Longitude': lon_values, 'Latitude': lat_values, 'tem': elevation_values})

# 关闭数据集
dataset = None
df = df[df['tem'] != 0]
# 打印DataFrame

dft = pd.merge(df, dfm, on=['Longitude', 'Latitude'], how='inner')
dft

C:\Users\lenovo\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Longitude,Latitude,tem,Unnamed: 0,rain
0,108.171251,33.742821,10.243101,354,562.75810
1,108.171251,33.741148,10.258933,355,578.33860
2,108.171251,33.739474,10.268854,356,593.80176
3,108.171251,33.737801,10.273345,357,608.89874
4,108.171251,33.736128,10.272860,358,623.36910
...,...,...,...,...,...
16033,108.489183,33.471327,10.250770,28482,695.17030
16034,108.489183,33.469649,10.293420,28483,689.45990
16035,108.489183,33.467970,10.320899,28484,684.58154
16036,108.489183,33.466292,10.331690,28485,680.56530


In [38]:
dft = dft.drop('Unnamed: 0', axis=1)
dft.to_csv("降水+气温.csv")
dft

,Longitude,Latitude,tem,rain
0,108.171251,33.742821,10.243101,562.75810
1,108.171251,33.741148,10.258933,578.33860
2,108.171251,33.739474,10.268854,593.80176
3,108.171251,33.737801,10.273345,608.89874
4,108.171251,33.736128,10.272860,623.36910
...,...,...,...,...
16033,108.489183,33.471327,10.250770,695.17030
16034,108.489183,33.469649,10.293420,689.45990
16035,108.489183,33.467970,10.320899,684.58154
16036,108.489183,33.466292,10.331690,680.56530


In [56]:
dfm=pd.read_csv("降水+气温.csv")
dfm['Longitude'] = dfm['Longitude'].round(2)
dfm['Latitude'] = dfm['Latitude'].round(2)
dfm

,Unnamed: 0,Longitude,Latitude,tem,rain
0,0,108.17,33.74,10.243101,562.75810
1,1,108.17,33.74,10.258933,578.33860
2,2,108.17,33.74,10.268854,593.80176
3,3,108.17,33.74,10.273345,608.89874
4,4,108.17,33.74,10.272860,623.36910
...,...,...,...,...,...
16033,16033,108.49,33.47,10.250770,695.17030
16034,16034,108.49,33.47,10.293420,689.45990
16035,16035,108.49,33.47,10.320899,684.58154
16036,16036,108.49,33.47,10.331690,680.56530


In [57]:
dfm = dfm.drop('Unnamed: 0', axis=1)
# 加载DEM栅格数据
dem_path = 'dem1.tif'
dataset = gdal.Open(dem_path)

# 获取DEM栅格的宽度和高度
cols = dataset.RasterXSize
rows = dataset.RasterYSize

# 获取DEM栅格的地理转换信息
transform = dataset.GetGeoTransform()

# 创建投影转换器
src_proj = pyproj.Proj(dataset.GetProjection())
target_proj = pyproj.Proj(init='EPSG:4326')  # WGS84坐标系
transformer = pyproj.Transformer.from_proj(src_proj, target_proj)

# 计算像元的经度和纬度
lon_values = []
lat_values = []

for i in range(cols):
    for j in range(rows):
        # 计算像元的中心坐标
        x = transform[0] + (i + 0.5) * transform[1] + (j + 0.5) * transform[2]
        y = transform[3] + (i + 0.5) * transform[4] + (j + 0.5) * transform[5]

        # 进行坐标转换
        lon, lat = transformer.transform(x, y)

        lon_values.append(lon)
        lat_values.append(lat)

# 获取DEM栅格的海拔值
band = dataset.GetRasterBand(1)
elevation_values = band.ReadAsArray().flatten()

# 创建DataFrame
df = pd.DataFrame({'Longitude': lon_values, 'Latitude': lat_values, 'dem': elevation_values})

# 关闭数据集
dataset = None
df = df[df['dem'] != 0]
# 打印DataFrame
df['Longitude'] = df['Longitude'].round(2)
df['Latitude'] = df['Latitude'].round(2)
dft = pd.merge(dfm, df, on=['Longitude', 'Latitude'], how='inner')
dft = dft.groupby(['Longitude', 'Latitude']).mean().reset_index()
dft = dft.drop_duplicates()
dft

C:\Users\lenovo\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Longitude,Latitude,tem,rain,dem
0,108.17,33.46,10.403170,713.741940,1950.103035
1,108.17,33.47,10.333575,767.954903,1947.888633
2,108.17,33.48,8.691312,833.169150,1967.152542
3,108.17,33.49,7.801497,795.112190,1988.559431
4,108.17,33.50,7.491377,657.651438,2006.862517
...,...,...,...,...,...
952,108.49,33.70,8.894662,737.263357,1740.224378
953,108.49,33.71,8.185500,745.317328,1733.225532
954,108.49,33.72,8.597057,722.921574,1709.274106
955,108.49,33.73,10.207662,706.886827,1687.487257


In [58]:
dft.to_csv("气温+降水+海拔.csv")

In [59]:
# 加载DEM栅格数据
dem_path = 'aspect.tif'
dataset = gdal.Open(dem_path)

# 获取DEM栅格的宽度和高度
cols = dataset.RasterXSize
rows = dataset.RasterYSize

# 获取DEM栅格的地理转换信息
transform = dataset.GetGeoTransform()

# 创建投影转换器
src_proj = pyproj.Proj(dataset.GetProjection())
target_proj = pyproj.Proj(init='EPSG:4326')  # WGS84坐标系
transformer = pyproj.Transformer.from_proj(src_proj, target_proj)

# 计算像元的经度和纬度
lon_values = []
lat_values = []

for i in range(cols):
    for j in range(rows):
        # 计算像元的中心坐标
        x = transform[0] + (i + 0.5) * transform[1] + (j + 0.5) * transform[2]
        y = transform[3] + (i + 0.5) * transform[4] + (j + 0.5) * transform[5]

        # 进行坐标转换
        lon, lat = transformer.transform(x, y)

        lon_values.append(lon)
        lat_values.append(lat)

# 获取DEM栅格的海拔值
band = dataset.GetRasterBand(1)
elevation_values = band.ReadAsArray().flatten()

# 创建DataFrame
df = pd.DataFrame({'Longitude': lon_values, 'Latitude': lat_values, 'aspect': elevation_values})

# 关闭数据集
dataset = None
df = df[df['aspect'] != 0]
# 打印DataFrame
df['Longitude'] = df['Longitude'].round(2)
df['Latitude'] = df['Latitude'].round(2)
dfk = pd.merge(dft, df, on=['Longitude', 'Latitude'], how='inner')
dfk = dfk.groupby(['Longitude', 'Latitude']).mean().reset_index()
dfk = dfk.drop_duplicates()
dfk

C:\Users\lenovo\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Longitude,Latitude,tem,rain,dem,aspect
0,108.17,33.46,10.403170,713.741940,1950.103035,176.995117
1,108.17,33.47,10.333575,767.954903,1947.888633,168.402512
2,108.17,33.48,8.691312,833.169150,1967.152542,170.620819
3,108.17,33.49,7.801497,795.112190,1988.559431,175.556107
4,108.17,33.50,7.491377,657.651438,2006.862517,175.836746
...,...,...,...,...,...,...
952,108.49,33.70,8.894662,737.263357,1740.224378,185.920868
953,108.49,33.71,8.185500,745.317328,1733.225532,180.281189
954,108.49,33.72,8.597057,722.921574,1709.274106,175.190643
955,108.49,33.73,10.207662,706.886827,1687.487257,174.740479


In [60]:
dfk.to_csv("气温+降水+海拔+坡向.csv")

In [65]:
import geopandas as gpd
from shapely.geometry import Point

# 假设 shapefile 文件名为 shapefile.shp
# 请将路径替换为您实际的文件路径
shapefile_path = 'intersectt.shp'

# 读取面数据的shapefile文件
gdf = gpd.read_file(shapefile_path)

# 假设 dataframe 是包含经度（'Longitude'）和纬度（'Latitude'）字段的 DataFrame
# df = ...

# 创建一个空的新字段 "适宜" 并初始化为 "不适宜"
dfk['适宜'] = '不适宜'

# 遍历 DataFrame 的经纬度
for index, row in dfk.iterrows():
    point = Point(row['Longitude'], row['Latitude'])
    for geometry in gdf['geometry']:
        if geometry.geom_type == 'Polygon':
            if point.within(geometry):
                dfk.at[index, '适宜'] = '适宜'
                break
        elif geometry.geom_type == 'MultiPolygon':
            for polygon in geometry.geoms:
                if point.within(polygon):
                    dfk.at[index, '适宜'] = '适宜'
                    break

# 打印修改后的 DataFrame
print(dfk)

     Longitude  Latitude        tem        rain          dem      aspect   适宜
0       108.17     33.46  10.403170  713.741940  1950.103035  176.995117  不适宜
1       108.17     33.47  10.333575  767.954903  1947.888633  168.402512  不适宜
2       108.17     33.48   8.691312  833.169150  1967.152542  170.620819  不适宜
3       108.17     33.49   7.801497  795.112190  1988.559431  175.556107  不适宜
4       108.17     33.50   7.491377  657.651438  2006.862517  175.836746   适宜
..         ...       ...        ...         ...          ...         ...  ...
952     108.49     33.70   8.894662  737.263357  1740.224378  185.920868  不适宜
953     108.49     33.71   8.185500  745.317328  1733.225532  180.281189  不适宜
954     108.49     33.72   8.597057  722.921574  1709.274106  175.190643  不适宜
955     108.49     33.73  10.207662  706.886827  1687.487257  174.740479   适宜
956     108.49     33.74  10.411073  701.620560  1656.807903  175.643906  不适宜

[957 rows x 7 columns]


In [66]:
dfk.to_csv("气温+降水+海拔+坡向+适宜.csv")